<a href="https://colab.research.google.com/github/Seligmann/carvana-image-masking-challenge/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing 
import os
import PIL
import numpy as np
import pathlib
import pandas as pd

import tensorflow_datasets as tfds

In [31]:
from IPython.display import clear_output
import matplotlib.pyplot as plt

In [32]:
import os

# # Upload the API token.
# def get_kaggle_credentials():
#   token_dir = os.path.join(os.path.expanduser("~"),".kaggle")
#   token_file = os.path.join(token_dir, "kaggle.json")
#   if not os.path.isdir(token_dir):
#     os.mkdir(token_dir)
#   try:
#     with open(token_file,'r') as f:
#       pass
#   except IOError as no_file:
#     try:
#       from google.colab import files
#     except ImportError:
#       raise no_file
    
#     uploaded = files.upload()
    
#     if "kaggle.json" not in uploaded:
#       raise ValueError("You need an API key! see: "
#                        "https://github.com/Kaggle/kaggle-api#api-credentials")
#     with open(token_file, "wb") as f:
#       f.write(uploaded["kaggle.json"])
#     os.chmod(token_file, 600)

# get_kaggle_credentials()

In [33]:
# import kaggle

In [34]:
# Download competition data to /content
# !kaggle competitions download -c carvana-image-masking-challenge

In [35]:
metadata_csv = '/content/drive/MyDrive/MyDrive/carvana-image-masking-challenge/metadata.csv'
train_masks_csv = '/content/drive/MyDrive/MyDrive/carvana-image-masking-challenge/train_masks.csv'
train_data = '/content/drive/MyDrive/MyDrive/carvana-image-masking-challenge/images/train'
train_masks_data = '/content/drive/MyDrive/MyDrive/carvana-image-masking-challenge/images/train_masks'
test_data = '/content/drive/MyDrive/MyDrive/carvana-image-masking-challenge/images/test'

In [36]:
metadata_df = pd.read_csv(metadata_csv)
train_masks_df = pd.read_csv(train_masks_csv)

In [37]:
metadata_df.head()

,id,year,make,model,trim1,trim2
0,0004d4463b50,2014.0,Acura,TL,TL,w/SE
1,00087a6bd4dc,2014.0,Acura,RLX,RLX,w/Tech
2,000aa097d423,2012.0,Mazda,MAZDA6,MAZDA6,i Sport
3,000f19f6e7d4,2016.0,Chevrolet,Camaro,Camaro,SS
4,00144e887ae9,2015.0,Acura,TLX,TLX,SH-AWD V6 w/Advance Pkg


In [38]:
train_masks_df.head()

,img,rle_mask
0,00087a6bd4dc_01.jpg,879386 40 881253 141 883140 205 885009 17 8850...
1,00087a6bd4dc_02.jpg,873779 4 875695 7 877612 9 879528 12 881267 15...
2,00087a6bd4dc_03.jpg,864300 9 866217 13 868134 15 870051 16 871969 ...
3,00087a6bd4dc_04.jpg,879735 20 881650 26 883315 92 883564 30 885208...
4,00087a6bd4dc_05.jpg,883365 74 883638 28 885262 119 885550 34 88716...


In [39]:
train_files = os.listdir(train_data)
train_mask_files = os.listdir(train_masks_data)
test_files = os.listdir(test_data)
print("Train files count:", len(train_files))
print("Train mask files count:", len(train_mask_files))
print("Test files count:", len(test_files))

Train files count: 5088
Train mask files count: 5088
Test files count: 100064


In [40]:
# metadata_df.rename({'image_id':'image_name'}, axis=1, inplace=True) # ? Pretty sure this was already done
# metadata_df.head()

In [41]:
# Get unique ids for each of the photos in train and test datasets
train_ids = []
for n in train_files:
  train_ids.append(n.split("_")[0])

test_ids = []
for n in test_files:
  test_ids.append(n.split("_")[0])

train_mask_ids = []
for n in train_mask_files:
  train_mask_ids.append(n.split("_")[0])

In [66]:
def GetCarImageFiles(car_image_id, get_mask=False):
  if get_mask:
    if car_image_id in masks_ids: # car id in mask ids?
      for n in train_mask_files:
        if car_image_id in n: # car id in training mask id?
          return [train_masks_data + "/" + n]

def GetImageMatrix(image_path):
  img = Image.open(image_path)
  return np.asarray(img, dtype=np.uint8)

['/content/drive/MyDrive/MyDrive/carvana-image-masking-challenge/images/train/cf65b1c5e147_09.jpg', '/content/drive/MyDrive/MyDrive/carvana-image-masking-challenge/images/train/cf65b1c5e147_10.jpg', '/content/drive/MyDrive/MyDrive/carvana-image-masking-challenge/images/train/cf65b1c5e147_11.jpg', '/content/drive/MyDrive/MyDrive/carvana-image-masking-challenge/images/train/cf65b1c5e147_12.jpg', '/content/drive/MyDrive/MyDrive/carvana-image-masking-challenge/images/train/cf65b1c5e147_13.jpg', '/content/drive/MyDrive/MyDrive/carvana-image-masking-challenge/images/train/cf65b1c5e147_14.jpg', '/content/drive/MyDrive/MyDrive/carvana-image-masking-challenge/images/train/cf65b1c5e147_15.jpg', '/content/drive/MyDrive/MyDrive/carvana-image-masking-challenge/images/train/cf65b1c5e147_16.jpg', '/content/drive/MyDrive/MyDrive/carvana-image-masking-challenge/images/train/cf65b1c5e147_01.jpg', '/content/drive/MyDrive/MyDrive/carvana-image-masking-challenge/images/train/cf65b1c5e147_02.jpg', '/content